https://medium.com/@cheng3374/ปรับ-parameter-ให้-model-มีประสิทธิภาพด้วย-gridsearch-32174da8da95

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
dataSet = pd.read_csv("Alzheimer.csv")
dataSet

,HN,Education,age,hypertension,heart_disease,family history,Occupation,Triglycerides,Blood Sugar,BMI,smoking_status,alzheimer
0,11046,Secondary School,67.0,0,1,Yes,Private,Normal,228.69,36.6,formerly smoked,1
1,51676,Bachelor,61.0,0,0,Yes,Freelance,High,202.21,NaN,never smoked,1
2,35112,Secondary School,80.0,0,1,Yes,Private,High,105.92,32.5,never smoked,1
3,67182,Bachelor,49.0,0,0,Yes,Private,Normal,171.23,34.4,smokes,1
4,1665,Bachelor,79.0,1,0,Yes,Freelance,High,174.12,24.0,never smoked,1
...,...,...,...,...,...,...,...,...,...,...,...,...
4095,68398,Secondary School,82.0,1,0,Yes,Freelance,High,71.97,28.3,never smoked,0
4096,14180,Bachelor,13.0,0,0,No,Farmer,High,103.08,18.6,Unknown,0
4097,44873,Bachelor,81.0,0,0,Yes,Freelance,Normal,125.20,40.0,never smoked,0
4098,19723,Bachelor,35.0,0,0,Yes,Freelance,High,82.99,30.6,never smoked,0


# MARK: Data Processing

In [3]:
dataSet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4100 entries, 0 to 4099
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   HN              4100 non-null   int64  
 1   Education       4100 non-null   object 
 2   age             4100 non-null   float64
 3   hypertension    4100 non-null   int64  
 4   heart_disease   4100 non-null   int64  
 5   family history  4100 non-null   object 
 6   Occupation      4100 non-null   object 
 7   Triglycerides   4100 non-null   object 
 8   Blood Sugar     4100 non-null   float64
 9   BMI             3936 non-null   float64
 10  smoking_status  4100 non-null   object 
 11  alzheimer       4100 non-null   int64  
dtypes: float64(3), int64(4), object(5)
memory usage: 384.5+ KB


In [4]:
dataSet.isna().sum()

HN                  0
Education           0
age                 0
hypertension        0
heart_disease       0
family history      0
Occupation          0
Triglycerides       0
Blood Sugar         0
BMI               164
smoking_status      0
alzheimer           0
dtype: int64

### MARK: เลือกที่จะ drop ทิ้งเพราะว่า Null มีจำนวนไม่เยอะมาก ไม่มีผลต่อ data เท่าที่ควร และ ไม่สามารถหาค่า BMI มาแทนที่ได้เพราะไม่มี weight และ height ของผู้ป่วย

In [5]:
dataSet = dataSet.dropna()
dataSet.isna().sum()

HN                0
Education         0
age               0
hypertension      0
heart_disease     0
family history    0
Occupation        0
Triglycerides     0
Blood Sugar       0
BMI               0
smoking_status    0
alzheimer         0
dtype: int64

### MARK: ตรวจสอบ Class Target จำนวนเหมาะสมกันหรือไม่

In [6]:
print("Target == 1 ->", len(dataSet.loc[dataSet["alzheimer"] == 1, "alzheimer"]))

Target == 1 -> 202


In [7]:
print("Target == 1 ->", len(dataSet.loc[dataSet["alzheimer"] == 0, "alzheimer"]))

Target == 1 -> 3734


### MARK: ปรับข้อมูล เป็น 0,1 <- LabelEncoder

In [8]:
from sklearn.preprocessing import LabelEncoder

In [9]:
dataSet.head()

,HN,Education,age,hypertension,heart_disease,family history,Occupation,Triglycerides,Blood Sugar,BMI,smoking_status,alzheimer
0,11046,Secondary School,67.0,0,1,Yes,Private,Normal,228.69,36.6,formerly smoked,1
2,35112,Secondary School,80.0,0,1,Yes,Private,High,105.92,32.5,never smoked,1
3,67182,Bachelor,49.0,0,0,Yes,Private,Normal,171.23,34.4,smokes,1
4,1665,Bachelor,79.0,1,0,Yes,Freelance,High,174.12,24.0,never smoked,1
5,66889,Secondary School,81.0,0,0,Yes,Private,Normal,186.21,29.0,formerly smoked,1


In [10]:
label_encoder = LabelEncoder()

In [11]:
dataSet['Education'] = label_encoder.fit_transform(dataSet['Education'])
dataSet['family history'] = label_encoder.fit_transform(dataSet['family history'])
dataSet['Occupation'] = label_encoder.fit_transform(dataSet['Occupation'])
dataSet['Triglycerides'] = label_encoder.fit_transform(dataSet['Triglycerides'])
dataSet['smoking_status'] = label_encoder.fit_transform(dataSet['smoking_status'])

/var/folders/gc/m_ppj8w50ggfmxvn8lf4jwxm0000gp/T/ipykernel_89429/447424765.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataSet['Education'] = label_encoder.fit_transform(dataSet['Education'])
/var/folders/gc/m_ppj8w50ggfmxvn8lf4jwxm0000gp/T/ipykernel_89429/447424765.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataSet['family history'] = label_encoder.fit_transform(dataSet['family history'])
/var/folders/gc/m_ppj8w50ggfmxvn8lf4jwxm0000gp/T/ipykernel_89429/447424765.py:3: SettingWithCopyWarnin

In [12]:
dataSet

,HN,Education,age,hypertension,heart_disease,family history,Occupation,Triglycerides,Blood Sugar,BMI,smoking_status,alzheimer
0,11046,1,67.0,0,1,1,4,1,228.69,36.6,1,1
2,35112,1,80.0,0,1,1,4,0,105.92,32.5,2,1
3,67182,0,49.0,0,0,1,4,1,171.23,34.4,3,1
4,1665,0,79.0,1,0,1,1,0,174.12,24.0,2,1
5,66889,1,81.0,0,0,1,4,1,186.21,29.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
4095,68398,1,82.0,1,0,1,1,0,71.97,28.3,2,0
4096,14180,0,13.0,0,0,0,0,0,103.08,18.6,0,0
4097,44873,0,81.0,0,0,1,1,1,125.20,40.0,2,0
4098,19723,0,35.0,0,0,1,1,0,82.99,30.6,2,0


# MARK: แบ่งข้อมูล

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

### MARK: train_test_split

In [14]:
X = dataSet.drop('alzheimer', axis=1)
y = dataSet["alzheimer"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [15]:
X_train.head()

,HN,Education,age,hypertension,heart_disease,family history,Occupation,Triglycerides,Blood Sugar,BMI,smoking_status
2837,22939,0,22.0,0,0,0,4,0,80.72,29.3,0
2069,464,1,46.0,0,0,1,4,0,78.44,23.9,2
1724,21724,0,42.0,0,0,1,1,1,124.34,34.7,1
1826,2868,0,54.0,0,0,1,3,0,102.61,32.4,0
514,42359,1,9.0,0,0,0,0,1,122.22,17.7,0


In [16]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2755 entries, 2837 to 3323
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   HN              2755 non-null   int64  
 1   Education       2755 non-null   int64  
 2   age             2755 non-null   float64
 3   hypertension    2755 non-null   int64  
 4   heart_disease   2755 non-null   int64  
 5   family history  2755 non-null   int64  
 6   Occupation      2755 non-null   int64  
 7   Triglycerides   2755 non-null   int64  
 8   Blood Sugar     2755 non-null   float64
 9   BMI             2755 non-null   float64
 10  smoking_status  2755 non-null   int64  
dtypes: float64(3), int64(8)
memory usage: 258.3 KB


In [17]:
y_train.info()

<class 'pandas.core.series.Series'>
Int64Index: 2755 entries, 2837 to 3323
Series name: alzheimer
Non-Null Count  Dtype
--------------  -----
2755 non-null   int64
dtypes: int64(1)
memory usage: 43.0 KB


### SMOTE (Synthetic Minority Over-sampling Technique): วิธีการที่ใช้การสร้างตัวอย่างสัญญาณเทียบเท่าจากคลาสน้อยโดยการผสมคุณลักษณะของตัวอย่างในคลาสน้อย

In [18]:
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler

In [19]:
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

In [20]:
from collections import Counter
print("จำนวนตัวอย่างของแต่ละคลาสก่อน SMOTE:", Counter(y_train))
print("จำนวนตัวอย่างของแต่ละคลาสหลังจาก SMOTE:", Counter(y_train_smote))

จำนวนตัวอย่างของแต่ละคลาสก่อน SMOTE: Counter({0: 2620, 1: 135})
จำนวนตัวอย่างของแต่ละคลาสหลังจาก SMOTE: Counter({0: 2620, 1: 2620})


## MARK: ทำ scaler
https://medium.com/data-espresso/scale-or-standardize-or-normalize-ef905613f275

In [21]:
scaler = StandardScaler()

In [22]:
X_train_scaled = scaler.fit_transform(X_train_smote)
X_test_scaled = scaler.transform(X_test)

# Model Neural Network

In [23]:
from sklearn.neural_network import MLPClassifier

In [24]:
nnModel = MLPClassifier(hidden_layer_sizes=(10, 5), max_iter=1000, random_state=42)

In [25]:
nnModel.fit(X_train_scaled, y_train_smote)

MLPClassifier(hidden_layer_sizes=(10, 5), max_iter=1000, random_state=42)

In [26]:
print("ตัวอย่างข้อมูลเรียนรู้ ->",nnModel.score(X_train_scaled,y_train_smote))

ตัวอย่างข้อมูลเรียนรู้ -> 0.8860687022900763


In [27]:
y_pred = nnModel.predict(X_test_scaled)
print("ตัวอย่างข้อมูลชุดทดสอบ ->",accuracy_score(y_test, y_pred))

ตัวอย่างข้อมูลชุดทดสอบ -> 0.7891617273497037


In [28]:
pd.crosstab(y_test, y_pred)

col_0,0,1
alzheimer,,
0,903,211
1,38,29


In [29]:
print("Classification Report:\n", classification_report(y_test, y_pred))

Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.81      0.88      1114
           1       0.12      0.43      0.19        67

    accuracy                           0.79      1181
   macro avg       0.54      0.62      0.53      1181
weighted avg       0.91      0.79      0.84      1181



## MARK: หา Best Parameter

In [30]:
from sklearn.model_selection import GridSearchCV

In [31]:
param_grid = {
    'hidden_layer_sizes': [(24,), (24, 24), (12, 24), (24, 12)],
    'activation': ['logistic', 'relu'],
    'solver': ['adam'],
     'max_iter': [500, 700, 1000],
    'learning_rate': ['constant', 'invscaling', 'adaptive'],
    'batch_size':[16,32,64]
}

In [32]:
mlp = MLPClassifier(random_state=42)

In [ ]:
grid_search = GridSearchCV(mlp, param_grid, cv=10, scoring='accuracy')
grid_result = grid_search.fit(X_train_scaled, y_train_smote)
grid_result

/Users/dev/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/dev/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/dev/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/dev/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/dev/anaconda3/lib/python3.11/site-pac

/Users/dev/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/dev/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/dev/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/dev/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/dev/anaconda3/lib/python3.11/site-pac

In [ ]:
best_params = grid_search.best_params_
print("Best Parameters:", best_params)

In [ ]:
best_accuracy = grid_search.best_score_
print("Best Accuracy:", best_accuracy)

In [ ]:
nn_model = MLPClassifier(**best_params, random_state=42) 
# nn_model = MLPClassifier(**best_params, max_iter=1000, random_state=42)

In [ ]:
nn_model.fit(X_train_scaled, y_train_smote)

In [ ]:
print("ตัวอย่างข้อมูลเรียนรู้ ->",nn_model.score(X_train_scaled,y_train_smote))

In [ ]:
y_pred = nn_model.predict(X_test_scaled)
print("ตัวอย่างข้อมูลชุดทดสอบ ->",accuracy_score(y_test, y_pred))

In [ ]:
pd.crosstab(y_test, y_pred)

In [ ]:
print("Classification Report:\n", classification_report(y_test, y_pred))